In [2]:
import pandas as pd
data = pd.read_csv('./ner_tagged.tsv', sep='\t')

In [3]:
data.rename(columns = {'Unnamed: 3':'Tags','-DOCSTART-':'Tokens','-X-':'X'}, inplace = True)

In [11]:
!python3 -m pip install --user pyspellchecker
from spellchecker import SpellChecker

spell2 = SpellChecker()

In [25]:
import csv
#Create attributes file from https://nijianmo.github.io/amazon/index.html
known_attributes = set()
with open('attributes.csv', newline='\n') as csvfile:
    attributes = csv.reader(csvfile, delimiter=' ')
    for row in attributes:
        for attr in row:
            known_attributes.add(attr.lower())

In [26]:
known_attributes

{'l', 'm', 's', 'unflavored', 'xl', 'xs'}

In [27]:
neg_sents = []
current_sent = []
current_tags = []
contains_neg = False
non_neg = 0
all_sents = []

for i, row in data.reset_index().iterrows():
    if type(row.Tags) == str:
        corrected = row.Tokens
        if row.Tokens.isalpha() and row.Tokens.lower() not in known_attributes:
            corrected = spell2.correction(row.Tokens)#spell(row.Tokens)
        if corrected != row.Tokens:
            print(f"{row.Tokens} changed to {corrected}")
        current_sent.append((row.Tokens, row.Tags))
        if "-N-" in row.Tags:
            contains_neg = True
    else:
        all_sents.append(current_sent)
        current_sent = []
        contains_neg = False

# all_sents

fireclay changed to foreplay
Karbolyn changed to carolyn
w changed to i
x changed to i
curbless changed to cureless
twik changed to twin
Touchless changed to touches
frameless changed to shameless
buscuit changed to biscuit
goldish changed to goldfish
suger changed to super
proteine changed to protein
ketosis changed to mitosis
supplment changed to supplement
tuque changed to toque
simiar changed to similar
mm changed to my
couter changed to outer
sinc changed to since
bathrub changed to bathtub
seatings changed to seating
Frameless changed to shameless
x changed to i
bathub changed to bathtub
f changed to i
frameless changed to shameless


In [28]:
for i in range(len(all_sents)):
    print(" ".join(word for word, tag in all_sents[i]))
    print()
#     print(" ".join(word + "-" + tag for word, tag in all_sents[i]))


I am looking for a black gloss 33 inch fireclay apron sink

Looking for pre workout Pump addict instead of Karbolyn Hydrate Wich one is better?

i need a 48 inch glass sliding goof and a shower pan system for 500.99 or less

Hello, do any of your free standing tubs have a matte white finish instead of a glossy finish?

I'm looking for a 24 inch white mirror that is freestanding and reasonably priced.

Does the linen add cotton duvet cover come in King? I saw it in the store but I don't see the king size online ?

Hi there. I'm looking to buy some throw blankets. Your website states that the underside of the Boucle wool-like throw is very soft. I would like to know if the wool-like top side is soft as well, or if it is scratchy. I have hypersensitive skin and need a blanket that will not irritate me. Thanks.

Long Sherpa jacket dark brown why can't I find it?

Hello. I'm looking for a soft Open face cardigan w no buttons.

hi do you carry any oval tubs in a matte white finish or not too